In [1]:
from sklearn.svm import LinearSVC
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import sys
import argparse
import csv
from scipy.sparse import *
import pandas as pd
import regex 
from sklearn.base import TransformerMixin
from nltk.tokenize import TreebankWordTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.util import ngrams
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
import gensim

In [2]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation,GRU
from keras.layers import Bidirectional, GlobalMaxPool1D,Concatenate,Conv1D,SpatialDropout1D,BatchNormalization,concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
import numpy as np


In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [5]:
from tensorflow.keras.layers import (TimeDistributed, Dropout, BatchNormalization, Flatten, Convolution1D, Activation, Input, Dense, LSTM, Lambda, Bidirectional,
                                     Add, AveragePooling1D, Multiply, GRU, GRUCell, LSTMCell, SimpleRNNCell, SimpleRNN, TimeDistributed, RNN,
                                     RepeatVector, Conv1D, MaxPooling1D, Concatenate, GlobalAveragePooling1D, UpSampling1D)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy, mean_squared_error
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras import losses, models, optimizers
from tensorflow.keras import backend as K
import tensorflow as tf

In [6]:
!pip install tensorflow-addons


     |████████████████████████████████| 1.1 MB 4.0 MB/s 


In [7]:
import tensorflow_addons as tfa


In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [9]:
def encode_text_as_word_indexes(train_tweets, test_tweets,valid_tweets , max_num_words=None, lower=False, char_level=True):
    # Create the tokenizer
    tokenizer = Tokenizer(num_words=max_num_words, filters='', lower=lower, split=" ", char_level=char_level)
    # Fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_tweets)
    # Encode each example as a sequence of word indexes based on the vocabulary of the tokenizer
    x_train = tokenizer.texts_to_sequences(train_tweets)
    x_test=tokenizer.texts_to_sequences(test_tweets)
    x_valid=tokenizer.texts_to_sequences(valid_tweets)
   
    return tokenizer, x_train,x_test,x_valid

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
df=pd.read_csv('/content/drive/MyDrive/ISarcasm/DataSet/train.En.csv')

In [12]:
df_isarcasm_new=df.loc[df['sarcastic']==1]

In [13]:
df_isarcasm_new=df_isarcasm_new[['tweet','sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']]

In [14]:
df_isarcasm_new.dropna(inplace=True)

In [15]:
train, validate, test = \
              np.split(df_isarcasm_new.sample(frac=1, random_state=42), 
                       [int(.6*len(df_isarcasm_new)), int(.8*len(df_isarcasm_new))])

In [16]:
train_new=pd.concat([train, validate], ignore_index=True)

In [17]:
def count_character_type(text):
    text=str(text)

    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "!":
            specialChar+=1
        
    return specialChar
    
def count_question_mark(text):
    text=str(text)
    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "?":
            specialChar+=1
        
    return specialChar

The Python NLP library, TextBlob has a method to roughly quantify if a sentence is fact or opinion.¶
The method outputs a number ranging from 0 to 1, an output close to 0 indicates the sentence is highly factual and close to 1 means the sentence is highly subjective. Here, we take the sum for every comment, an overall higher sum then, will be indicative of higher subjectivity

In [18]:
def get_subjectivity(text):
    return TextBlob(str(text)).sentiment.subjectivity

The distribution of the 4 most important parts-of-speech (Noun, Verb, Adjective,Pronoun) in sarcastic and non sarcastic comments might shed some light, so let's plot it as a grouped bar chart

In [19]:
import spacy

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [22]:
def pos_counter(doc):
    verb_count = 0
    adj_count = 0
    pron_count = 0
    noun_count=0
    for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
        elif tok.pos_ == "ADJ":
            adj_count=adj_count+1
        elif tok.pos_ == "PRON":
            pron_count=pron_count+1
        elif tok.pos_ == "PROPN":
            noun_count=noun_count+1
    return (verb_count,adj_count,pron_count,noun_count)

In [23]:
def get_verb_count(text):
  verb_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
  return verb_count

def get_noun_count(text):
  noun_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PROPN":
            noun_count=noun_count+1
  return noun_count

def get_pron_count(text):
  pron_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PRON":
            pron_count=pron_count+1
  return pron_count


def get_adj_count(text):
  adj_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "ADJ":
            adj_count=adj_count+1
  return adj_count

Sarcastic comments tend to have profanity interspersed within, let's get a count of both sarcastic and non sarcastic and see how they compare

In [24]:
!pip install better_profanity 
from better_profanity import profanity

     |████████████████████████████████| 46 kB 1.3 MB/s 


In [25]:
def count_profane_words(text):
    count = 0
    for sent in str(text).split():
        if profanity.contains_profanity(sent) == True:
            count = count+1
    return count

get entity cloud

In [26]:
def entity_wordcloud(text):
    named_entities = []    
    sent = nlp(str(text))
    for ent in sent.ents:
        if ent.label_ == 'PERSON' or 'ORG' or 'GPE':
            named_entities.append(ent.text)
            
    doc = " ".join(named_entities)
    return doc

In [27]:
!git clone https://github.com/larsmans/sentiwordnet.git

Cloning into 'sentiwordnet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [28]:
import csv

In [29]:
class load_senti_word_net(object):
    """
    constructor to load the file and read the file as CSV
    6 columns - pos, ID, PosScore, NegScore, synsetTerms, gloss
    synsetTerms can have multiple similar words like abducting#1 abducent#1 and will read each one and calculaye the scores
    """
    
    def __init__(self):
        sent_scores = collections.defaultdict(list)
        with io.open("/content/sentiwordnet/SentiWordNet_3.0.0_20130122.txt") as fname:
            file_content = csv.reader(fname, delimiter='\t',quotechar='"')
            
            for line in file_content:                
                if line[0].startswith('#') :
                    continue                    
                pos, ID, PosScore, NegScore, synsetTerms, gloss = line
                for terms in synsetTerms.split(" "):
                    term = terms.split("#")[0]
                    term = term.replace("-","").replace("_","")
                    key = "%s/%s"%(pos,term.split("#")[0])
                    try:
                        sent_scores[key].append((float(PosScore),float(NegScore)))
                    except:
                        sent_scores[key].append((0,0))
                    
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value,axis=0)
        
        self.sent_scores = sent_scores    
     
    """
    For a word,
    nltk.pos_tag(["Suraj"])
    [('Suraj', 'NN')]
    """
    
    def score_word(self, word):
        pos = nltk.pos_tag([word])[0][1]
        return self.score(word, pos)
    
    def score(self,word, pos):
        """
        Identify the type of POS, get the score from the senti_scores and return the score
        """
        
        if pos[0:2] == 'NN':
            pos_type = 'n'
        elif pos[0:2] == 'JJ':
            pos_type = 'a'
        elif pos[0:2] =='VB':
            pos_type='v'
        elif pos[0:2] =='RB':
            pos_type = 'r'
        else:
            pos_type =  0
            
        if pos_type != 0 :    
            loc = pos_type+'/'+word
            score = self.sent_scores[loc]
            if len(score)>1:
                return score
            else:
                return np.array([0.0,0.0])
        else:
            return np.array([0.0,0.0])
        
    """
    Repeat the same for a sentence
    nltk.pos_tag(word_tokenize("My name is Suraj"))
    [('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Suraj', 'NNP')]    
    """    
        
    def score_sentencce(self, sentence):
        pos = nltk.pos_tag(sentence)
        # print (pos)
        mean_score = np.array([0.0, 0.0])
        for i in range(len(pos)):
            mean_score += self.score(pos[i][0], pos[i][1])
            
        return mean_score
    
    def pos_vector(self, sentence):
        pos_tag = nltk.pos_tag(sentence)
        vector = np.zeros(4)
        
        for i in range(0, len(pos_tag)):
            pos = pos_tag[i][1]
            if pos[0:2]=='NN':
                vector[0] += 1
            elif pos[0:2] =='JJ':
                vector[1] += 1
            elif pos[0:2] =='VB':
                vector[2] += 1
            elif pos[0:2] == 'RB':
                vector[3] += 1
                
        return vector
            

In [30]:
def gram_features(features,sentence):
    sentence_rep = replace_reg(str(sentence))
    token = nltk.word_tokenize(sentence_rep)
    token = [porter.stem(i.lower()) for i in token]        
    
    bigrams = nltk.bigrams(token)
    bigrams = [tup[0] + ' ' + tup[1] for tup in bigrams]
    grams = token + bigrams
    #print (grams)
    for t in grams:
        features['contains(%s)'%t]=1.0

In [31]:
def pos_features(features,sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)
    token = [ porter.stem(each.lower()) for each in token]
    pos_vector = sentiments.pos_vector(token)
    for j in range(len(pos_vector)):
        features['POS_'+str(j+1)] = pos_vector[j]
    # print ("done")
    

In [32]:
def capitalization(features,sentence):
    count = 0
    for i in range(len(sentence)):
        count += int(sentence[i].isupper())
    features['Capitalization'] = int(count > 3)
    # print (count)

In [33]:
import nltk
import re

#dictionnary to sentiment analysis
emo_repl = {
    #good emotions
    "&lt;3" : " good ",
    ":d" : " good ",
    ":dd" : " good ",
    ":p" : " good ",
    "8)" : " good ",
    ":-)" : " good ",
    ":)" : " good ",
    ";)" : " good ",
    "(-:" : " good ",
    "(:" : " good ",
    
    "yay!" : " good ",
    "yay" : " good ",
    "yaay" : " good ",
    "yaaay" : " good ",
    "yaaaay" : " good ",
    "yaaaaay" : " good ",    
    #bad emotions
    ":/" : " bad ",
    ":&gt;" : " sad ",
    ":')" : " sad ",
    ":-(" : " bad ",
    ":(" : " bad ",
    ":s" : " bad ",
    ":-s" : " bad "
}

#dictionnary for general (i.e. topic modeler)
emo_repl2 = {
    #good emotions
    "&lt;3" : " heart ",
    ":d" : " smile ",
    ":p" : " smile ",
    ":dd" : " smile ",
    "8)" : " smile ",
    ":-)" : " smile ",
    ":)" : " smile ",
    ";)" : " smile ",
    "(-:" : " smile ",
    "(:" : " smile ",
       
    #bad emotions
    ":/" : " worry ",
    ":&gt;" : " angry ",
    ":')" : " sad ",
    ":-(" : " sad ",
    ":(" : " sad ",
    ":s" : " sad ",
    ":-s" : " sad "
}

#general
re_repl = {
    r"\br\b" : "are",
    r"\bu\b" : "you",
    r"\bhaha\b" : "ha",
    r"\bhahaha\b" : "ha",
    r"\bdon't\b" : "do not",
    r"\bdoesn't\b" : "does not",
    r"\bdidn't\b" : "did not",
    r"\bhasn't\b" : "has not",
    r"\bhaven't\b" : "have not",
    r"\bhadn't\b" : "had not",
    r"\bwon't\b" : "will not",
    r"\bwouldn't\b" : "would not",
    r"\bcan't\b" : "can not",
    r"\bcannot\b" : "can not"    
}

emo_repl_order = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl.keys()]))]
emo_repl_order2 = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl2.keys()]))]

def replace_emo(sentence):
    sentence2 = sentence
    for k in emo_repl_order:
        sentence2 = sentence2.replace(k,emo_repl[k])
    for r, repl in re_repl.items():
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

def replace_reg(sentence):
    sentence2 = sentence
    for k in emo_repl_order2:
        sentence2 = sentence2.replace(k,emo_repl2[k])
    for r, repl in re_repl.items(): 
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

In [34]:
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

class topic(object):
    def __init__(self, nbtopic = 100, alpha=1,model=None,dicttp=None):
        self.nbtopic = nbtopic
        self.alpha = alpha
        self.porter = nltk.PorterStemmer()
        self.stop = stopwords.words('english')+['.','!','?','"','...','\\',"''",'[',']','~',"'m","'s",';',':','..','$']
        if model!=None and dicttp!=None:
            self.lda = models.ldamodel.LdaModel.load(model)
            self.dictionary =  corpora.Dictionary.load(dicttp)
            
    def fit(self,documents):
        
        documents_mod = documents
        tokens = [nltk.word_tokenize(sentence) for sentence in documents_mod]
        tokens = [[self.porter.stem(t.lower()) for t in sentence if t.lower() not in self.stop] for sentence in tokens]        
            
        self.dictionary = corpora.Dictionary(tokens)
        corpus = [self.dictionary.doc2bow(text) for text in tokens]
        self.lda = models.ldamodel.LdaModel(corpus,id2word=self.dictionary, num_topics=self.nbtopic,alpha=self.alpha)
        
        self.lda.save('topics.tp')
        self.dictionary.save('topics_dict.tp')
        
    def get_topic(self,topic_number):
        
        return self.lda.print_topic(topic_number)
    
    def transform(self,sentence):
        
        sentence_mod = sentence
        tokens = nltk.word_tokenize(sentence_mod)
        tokens = [self.porter.stem(t.lower()) for t in tokens if t.lower() not in self.stop] 
        corpus_sentence = self.dictionary.doc2bow(tokens)
        
        return self.lda[corpus_sentence] 

In [35]:
porter = nltk.PorterStemmer()


In [36]:
import string
def sentiment_extract(features, sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)    
    token = [porter.stem(i.lower()) for i in token]   
    mean_sentiment = sentiments.score_sentencce(token)
    features["Positive Sentiment"] = mean_sentiment[0]
    features["Negative Sentiment"] = mean_sentiment[1]
    features["sentiment"] = mean_sentiment[0] - mean_sentiment[1]
    #print(mean_sentiment[0], mean_sentiment[1])
    
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in token]).strip())
        features["Blob Polarity"] = text.sentiment.polarity
        features["Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["Blob Polarity"] = 0
        features["Blob Subjectivity"] = 0
        # print("do nothing")
        
    
    first_half = token[0:len(token)//2]    
    mean_sentiment_half = sentiments.score_sentencce(first_half)
    features["positive Sentiment first half"] = mean_sentiment_half[0]
    features["negative Sentiment first half"] = mean_sentiment_half[1]
    features["first half sentiment"] = mean_sentiment_half[0]-mean_sentiment_half[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in first_half]).strip())
        features["first half Blob Polarity"] = text.sentiment.polarity
        features["first half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["first Blob Polarity"] = 0
        features["first Blob Subjectivity"] = 0
        # print("do nothing")
    
    second_half = token[len(token)//2:]
    mean_sentiment_sechalf = sentiments.score_sentencce(second_half)
    features["positive Sentiment second half"] = mean_sentiment_sechalf[0]
    features["negative Sentiment second half"] = mean_sentiment_sechalf[1]
    features["second half sentiment"] = mean_sentiment_sechalf[0]-mean_sentiment_sechalf[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in second_half]).strip())
        features["second half Blob Polarity"] = text.sentiment.polarity
        features["second half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["second Blob Polarity"] = 0
        features["second Blob Subjectivity"] = 0
        # print("do nothing")  
    

In [37]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [38]:
topic_mod = topic(nbtopic=200,alpha='symmetric')
topic_mod.fit(train_new['tweet'].values)
# topic_mod = topic(model=os.path.join('topics.tp'),dicttp=os.path.join('topics_dict.tp'))
def topic_feature(features,sentence,topic_modeler):    
    topics = topic_modeler.transform(sentence)    
    for j in range(len(topics)):
        features['Topic :'] = topics[j][1]

In [39]:
!pip install emoji

     |████████████████████████████████| 170 kB 4.1 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=a30223bb6f6668a56c4e4a8043113ef8e4eeb03db5adcc4c3b3bf00532892040
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [40]:
from emoji import UNICODE_EMOJI



In [41]:
"""
Function to extract the below features.
1. Presence of emoji's
2. Count of number of Question marks
3. Count of number of Exclamation marks
4. presence of hashtags other than #sarcasm
5. presence of any @user tweets

"""
from collections import defaultdict

def Emoji_present(text):
    emoji = {
    "&lt;3" : "positive",":D" : "positive",	":d" : "positive", ":dd" : "positive", ":P" : "positive", ":p" : "positive","8)" : "positive",
    "8-)" : "positive",  ":-)" : "positive",    ":)" : "positive",    ";)" : "positive",    "(-:" : "positive",    "(:" : "positive",
    ":')" : "positive",    "xD" : "positive",    "XD" : "positive",  "yay!" : "positive",  "yay" : "positive",  "yaay" : "positive",
    "yaaay" : "positive",  "yaaaay" : "positive", "yaaaaay" : "positive", "Yay!" : "positive", "Yay" : "positive", "Yaay" : "positive",
    "Yaaay" : "positive", "Yaaaay" : "positive", "Yaaaaay" : "positive",  ":/" : "negative", "&gt;" : "negative", ":'(" : "negative",
    ":-(" : "negative", ":(" : "negative", ":s" : "negative",":-s" : "negative","-_-" : "negative", "-.-" : "negative" }
    emoji_count =defaultdict(int) 
    isPresent=False    
        
    for word in str(text):
        if word in emoji:            
            isPresent = True

    
    count = 0
    for emoji in UNICODE_EMOJI:
        count += str(text).count(emoji)
        if count>=1:
          isPresent=True      

    
    return isPresent 



In [42]:
from collections import Counter, OrderedDict
import collections


In [43]:
import io

sentiments = load_senti_word_net()

def get_features(sentence, topic_modeler):
    features = {}
    # gram_features(features,sentence)
    pos_features(features,sentence)
    sentiment_extract(features, sentence)
    capitalization(features,sentence)
    topic_feature(features, sentence,topic_modeler)
    return features

In [44]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [45]:
train_new.dropna(inplace=True)

In [46]:
train_new

,tweet,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,All the shade i have been hearing about Ben Pl...,1.0,0.0,0.0,0.0,0.0,0.0
1,you would think the odds of me sitting next to...,1.0,0.0,0.0,0.0,0.0,0.0
2,Billy Gunn sorta relevant for the first time i...,1.0,0.0,0.0,1.0,0.0,0.0
3,"I see Dettol, Toilet Duck and Zoflora are all ...",1.0,0.0,0.0,0.0,0.0,1.0
4,@catboychika Aren't you happy I gave you less ...,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
688,omg my ships are gonna love what taylor swift ...,1.0,0.0,0.0,0.0,0.0,0.0
689,Maybe if I watched suits in year 12 I’d be stu...,1.0,0.0,0.0,0.0,0.0,0.0
690,"Me: ""Did you you find everything you were look...",1.0,0.0,0.0,0.0,0.0,1.0
691,Broke: doing assignments in order of due date\...,1.0,0.0,0.0,0.0,0.0,0.0


In [47]:
train_new[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[0].values

array([1., 0., 0., 0., 0., 0.])

In [82]:
import re
featureset=[]
for i in range(0,len(train_new)):
    # if i==2774or i==2775:
    #   continue
    temp = str(train_new["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    featureset.append((get_features(temp,topic_mod), train_new[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[i].values, train_new["tweet"][i]))

In [83]:
c = []
for i in range(0,len(featureset)):
    c.append(pd.DataFrame(featureset[i][0],index=[i]))
result = pd.concat(c)

In [84]:
result.insert(loc=0,column="label",value='0')
for i in range(0, len(featureset)):
    result["label"].loc[i] = featureset[i][1]   

result.insert(loc=0,column="text",value='0')
for i in range(0, len(featureset)):
    result["text"].loc[i] = featureset[i][2]  
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [51]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first Blob Polarity,first Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second Blob Polarity,second Blob Subjectivity,Capitalization,Topic :
0,All the shade i have been hearing about Ben Pl...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16.0,3.0,8.0,0.0,0.486475,0.366756,0.119719,0,0,0.204833,0.196862,0.007971,0,0,0.281642,0.169893,0.111748,0,0,1,0.956739
1,you would think the odds of me sitting next to...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,6.0,5.0,0.0,0.617476,0.491941,0.125535,0,0,0.343094,0.245032,0.098062,0,0,0.274382,0.246909,0.027473,0,0,0,0.947632
2,Billy Gunn sorta relevant for the first time i...,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0]",6.0,1.0,0.0,0.0,0.229167,0.000000,0.229167,0,0,0.000000,0.000000,0.000000,0,0,0.229167,0.000000,0.229167,0,0,0,0.875625
3,"I see Dettol, Toilet Duck and Zoflora are all ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]",7.0,0.0,5.0,1.0,0.111435,0.109261,0.002175,0,0,0.046875,0.036458,0.010417,0,0,0.064560,0.072802,-0.008242,0,0,1,0.917083
4,@catboychika Aren't you happy I gave you less ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5.0,1.0,3.0,1.0,0.041667,0.316667,-0.275000,0,0,0.000000,0.000000,0.000000,0,0,0.041667,0.316667,-0.275000,0,0,0,0.889444


In [85]:
result['contain_emoji']=result['text'].apply(Emoji_present)

In [86]:
result['count_special_chracter']=result['text'].apply(count_character_type)

In [87]:
result['count_question_marks']=result['text'].apply(count_question_mark)

In [55]:
from textblob import TextBlob

In [88]:
result['subjectivity']=result['text'].apply(get_subjectivity)

In [89]:
result['count_verbs']=result['text'].apply(get_verb_count)

In [90]:
result['count_nouns']=result['text'].apply(get_noun_count)

In [91]:
result['count_pronun']=result['text'].apply(get_pron_count)

In [92]:
result['count_adjct']=result['text'].apply(get_adj_count)

In [93]:
result['entities']=result['text'].apply(entity_wordcloud)

In [94]:
result['count_profane']=result['text'].apply(count_profane_words)

In [63]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first Blob Polarity,first Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second Blob Polarity,second Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane
0,All the shade i have been hearing about Ben Pl...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16.0,3.0,8.0,0.0,0.486475,0.366756,0.119719,0,0,0.204833,0.196862,0.007971,0,0,0.281642,0.169893,0.111748,0,0,1,0.956739,True,1,3,0.680000,2,4,4,2,Ben Platt @DearEvanHansen Grease 1,0
1,you would think the odds of me sitting next to...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,6.0,5.0,0.0,0.617476,0.491941,0.125535,0,0,0.343094,0.245032,0.098062,0,0,0.274382,0.246909,0.027473,0,0,0,0.947632,True,0,0,0.451389,4,0,3,5,three,1
2,Billy Gunn sorta relevant for the first time i...,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0]",6.0,1.0,0.0,0.0,0.229167,0.000000,0.229167,0,0,0.000000,0.000000,0.000000,0,0,0.229167,0.000000,0.229167,0,0,0,0.875625,False,0,0,0.616667,0,2,0,2,Billy Gunn first years,0
3,"I see Dettol, Toilet Duck and Zoflora are all ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]",7.0,0.0,5.0,1.0,0.111435,0.109261,0.002175,0,0,0.046875,0.036458,0.010417,0,0,0.064560,0.072802,-0.008242,0,0,1,0.917083,True,0,0,0.000000,3,5,1,0,Dettol Toilet Duck Zoflora,0
4,@catboychika Aren't you happy I gave you less ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5.0,1.0,3.0,1.0,0.041667,0.316667,-0.275000,0,0,0.000000,0.000000,0.000000,0,0,0.041667,0.316667,-0.275000,0,0,0,0.889444,True,0,0,0.533333,2,0,3,2,,0


In [64]:
len(test)

174

In [65]:
import re
feature_set_test=[]
for i, row in test.iterrows():
    # if i==0:
    #   continue
    temp = str(test["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    feature_set_test.append((get_features(temp,topic_mod), test[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[i].values,test['tweet'][i]))

In [66]:
c = []
for i in range(0,len(feature_set_test)):
    c.append(pd.DataFrame(feature_set_test[i][0],index=[i]))
result_test = pd.concat(c)

In [67]:
result_test.insert(loc=0,column="label",value='0')
for i in range(0, len(feature_set_test)):
    result_test["label"].loc[i] = feature_set_test[i][1]   

result_test.insert(loc=0,column="text",value='0')
for i in range(0, len(feature_set_test)):
    result_test["text"].loc[i] = feature_set_test[i][2]   
    
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [68]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,can't wait til a bunch of pudgy suburban dads ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,3.0,1.0,0.095455,0.871212,-0.775758,0.00,0.000000,0.000000,0.791667,-0.791667,0.00,0.000000,0.095455,0.079545,0.015909,0.0,0.0,0,0.196581
1,Why are the side effects of antidepressants so...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]",2.0,2.0,1.0,0.0,0.000000,0.104167,-0.104167,0.00,0.000000,0.000000,0.010417,-0.010417,0.00,0.000000,0.000000,0.104167,-0.104167,0.0,0.0,0,0.502500
2,I love to pick my modules on a first come firs...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,5.0,5.0,1.0,1.166667,0.375000,0.791667,0.25,0.347917,0.979167,0.250000,0.729167,0.25,0.347917,0.187500,0.125000,0.062500,0.0,0.0,0,0.194877
3,@RealCandaceO Ok so when does the anger and bi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",6.0,3.0,1.0,2.0,0.491815,1.409226,-0.917411,-0.10,0.400000,0.229167,0.645833,-0.416667,-0.10,0.350000,0.262649,0.763393,-0.500744,-0.1,0.5,1,0.259419
4,@AvatarOdin @aubviouslynot sounds like the ame...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,1.0,1.0,0.208333,0.083333,0.125000,0.20,0.200000,0.000000,0.000000,0.000000,0.40,0.400000,0.208333,0.083333,0.125000,0.0,0.0,0,0.143571


In [69]:
result_test['contain_emoji']=result_test['text'].apply(Emoji_present)

In [70]:
result_test['count_special_chracter']=result_test['text'].apply(count_character_type)

In [71]:
result_test['count_question_marks']=result_test['text'].apply(count_question_mark)

In [72]:
result_test['subjectivity']=result_test['text'].apply(get_subjectivity)

In [73]:
result_test['count_verbs']=result_test['text'].apply(get_verb_count)

In [74]:
result_test['count_nouns']=result_test['text'].apply(get_noun_count)

In [75]:
result_test['count_pronun']=result_test['text'].apply(get_pron_count)

In [76]:
result_test['count_adjct']=result_test['text'].apply(get_adj_count)

In [77]:
result_test['entities']=result_test['text'].apply(entity_wordcloud)

In [78]:
result_test['count_profane']=result_test['text'].apply(count_profane_words)

In [79]:
result_test.dropna(inplace=True)

In [80]:
result.dropna(inplace=True)

In [95]:
x_train_feat,y_train=result[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']],result[['label']]

In [96]:
x_test_feat,y_test=result_test[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']],result_test[['label']]

In [97]:
y_train_l=[]
for i in range(0,len(y_train.to_numpy())):
  y_train_l.append(y_train.to_numpy()[i][0].astype(int))
y_train_l=np.array(y_train_l)


In [98]:
y_test_l=[]
for i in range(0,len(y_test.to_numpy())):
  y_test_l.append(y_test.to_numpy()[i][0].astype(int))
y_test_l=np.array(y_test_l)

In [99]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2021-12-27 18:33:23--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-12-27 18:33:23--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-12-27 18:33:24--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [100]:
!unzip -q glove.6B.zip


In [101]:
EMBEDDING_DIM=300

In [102]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="/content/glove.6B.300d.txt", word2vec_output_file="/content/gensim_glove_vectors.txt")

(400000, 300)

In [103]:
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("/content/gensim_glove_vectors.txt", binary=False)

In [104]:
init_unk = True
var = None
weighted = True

In [105]:
import random
from random import seed
import math

In [106]:
# Calculate the variance of an embedding (like glove, word2vec, emoji2vec, etc)
# Used to sample new uniform distributions of vectors in the interval [-variance, variance]
def embedding_variance(vec_map):
    variance = np.sum([np.var(vec) for vec in vec_map.vectors]) / len(vec_map.vectors)
    return variance

In [107]:
# Get the tf-idf weighting scheme (used to measure the contribution of a word in a tweet => weighted sum of embeddings)
def get_tf_idf_weights(tweets, vec_map):
    df = {}
    for tw in tweets:
        words = set(tw.split())
        for word in words:
            if word not in df:
                df[word] = 0.0
            df[word] += 1.0
    idf = OrderedDict()
    for word in vec_map.vocab:
        n = 1.0
        if word in df:
            n += df[word]
        score = math.log(len(tweets) / float(n))
        idf[word] = score
    return idf

In [108]:
# Get the vec representation of a set of tweets based on a specified embedding (can be a word or emoji mapping)
def get_tweets_embeddings(tweets, vec_map, embedding_dim=100, init_unk=False, variance=None, weighted_average=True):
    # Get the variance of the embedding map
    if init_unk and variance is None:
        variance = embedding_variance(vec_map)
        print("Vector mappings have variance ", variance)
    # If set, calculate the tf-idf weight of each embedding, otherwise, no weighting (all weights are 1.0)
    if weighted_average:
        weights = get_tf_idf_weights(tweets, vec_map)
    else:
        weights = {k: 1.0 for k in vec_map.vocab}
    tw_emb = np.zeros((len(tweets), embedding_dim))
    for i, tw in enumerate(tweets):
        total_valid = 0
        for word in tw.split():
            embedding_vector=None 
            if word in vec_map.wv:
              embedding_vector = vec_map[word]
            if embedding_vector is not None:
                tw_emb[i] = tw_emb[i] + embedding_vector * weights[word]
                total_valid += 1
            elif init_unk:
                seed(1337603)
                tw_emb[i] = np.random.uniform(-variance, variance, size=(1, embedding_dim))
            # else:
            #    print("Not found: ", word)
        # Get the average embedding representation for this tweet
        tw_emb[i] /= float(max(total_valid, 1))
    return tw_emb

In [109]:
x_train_word_emb = get_tweets_embeddings(result['text'], glove_model, EMBEDDING_DIM,
                                               init_unk=init_unk, variance=var, weighted_average=weighted)

Vector mappings have variance  0.145287607421875


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [110]:
x_test_word_emb = get_tweets_embeddings(result_test['text'], glove_model, EMBEDDING_DIM,
                                               init_unk=init_unk, variance=var, weighted_average=weighted)

Vector mappings have variance  0.145287607421875


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [111]:
from sklearn.multioutput import MultiOutputClassifier

In [112]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [113]:
def print_statistics(y, y_pred):
    accuracy = hamming_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred,target_names=['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'estimator__C': C_range}
    linear_svm = MultiOutputClassifier(LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2'))
    # print(linear_svm.get_params().keys())
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'estimator__kernel': ['rbf'], 'estimator__C': C_range, 'estimator__gamma': gamma_range}
    nonlinear_svm = MultiOutputClassifier(SVC(class_weight=class_ratio))
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'estimator__C': C_range}
    log_regr = MultiOutputClassifier(LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2'))
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = MultiOutputClassifier(LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2'))
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/svm_model_bin.sav'
    # joblib.dump(svm, filename)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = MultiOutputClassifier(LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2'))
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
    # joblib.dump(regr, filename)


def random_forest(x_train, y_train, x_test, y_test, class_ratio='balanced'):
  print_model_title("Random Forest")
  rf = MultiOutputClassifier(RandomForestClassifier(n_estimators=400, random_state=11))
  rf.fit(x_train, y_train)
  y_hat = rf.predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost")
  xgb_model =MultiOutputClassifier(XGBClassifier(max_depth=6, n_estimators=1000))
  xgb_model .fit(x_train, y_train)
  y_hat = xgb_model .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def xg_boost_focal_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Focal")
  xgboster_focal = imb_xgb(special_objective='focal')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost_weighted_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Weighted")
  xgboster_focal = imb_xgb(special_objective='weighted')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = MultiOutputClassifier(LinearSVC(C=0.1, penalty='l2'))
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)


def ensemble_stacked(x_train, y_train, x_test, y_test):
  print_model_title("Ensemble Stacked Classifiers")
  estimators = [ ('lr',MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'))),('svm_linear',MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2'))),('rf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10, random_state=42)))]
  from sklearn.ensemble import StackingClassifier
  clf = StackingClassifier(
      estimators=estimators )
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
def voting_classifiers(x_train, y_train, x_test, y_test,voting_type='hard'):
  print_model_title("Voting Classifier")
  estimators = [ ('lr',MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'))),('svm_linear',MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2'))),('rf',MultiOutputClassifier(RandomForestClassifier(n_estimators=10, random_state=42)))]
  from sklearn.ensemble import StackingClassifier
  clf = VotingClassifier(
      estimators=estimators , voting=voting_type)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
def Bagging_Classifier_LR(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier LR")
 
  clf =BaggingClassifier(base_estimator=MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def Bagging_Classifier_SVM(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier SVM")
 
  clf =BaggingClassifier(base_estimator=MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


from sklearn.ensemble import GradientBoostingClassifier
def gradient_boosting(x_train, y_train, x_test, y_test):
  print_model_title("Gradient Boosting")
 
  clf =MultiOutputClassifier(GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=30, random_state=42))
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)


In [114]:
from scipy.sparse import hstack, csr_matrix


In [118]:
train_features = hstack([
    x_train_word_emb
    ,csr_matrix(np.array(x_train_feat.to_numpy(), dtype=float))], 'csr'
)

In [120]:
test_features = hstack([
    x_test_word_emb,
    csr_matrix(np.array(x_test_feat.to_numpy(), dtype=float))], 'csr'
)

In [122]:
import warnings
warnings.filterwarnings('ignore')

In [123]:
linear_svm_grid(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.550
Precision: 0.638
Recall: 0.621
F_score: 0.627

                     precision    recall  f1-score   support

            sarcasm       0.80      0.76      0.78       138
              irony       0.29      0.28      0.28        36
             satire       0.17      0.20      0.18         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.08      0.33      0.13         6
rhetorical_question       0.50      0.52      0.51        25

          micro avg       0.57      0.62      0.59       211
          macro avg       0.31      0.35      0.31       211
       weighted avg       0.64      0.62      0.63       211
        samples avg       0.59      0.64      0.59       211



In [124]:
nonlinear_svm_grid(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.710
Precision: 0.692
Recall: 0.659
F_score: 0.590

                     precision    recall  f1-score   support

            sarcasm       0.80      1.00      0.89       138
              irony       1.00      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.80      0.66      0.72       211
          macro avg       0.30      0.17      0.16       211
       weighted avg       0.69      0.66      0.59       211
        samples avg       0.80      0.71      0.74       211



In [125]:
logistic_regression_grid(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.475
Precision: 0.595
Recall: 0.607
F_score: 0.595

                     precision    recall  f1-score   support

            sarcasm       0.78      0.73      0.75       138
              irony       0.15      0.17      0.16        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.08      0.33      0.13         6
rhetorical_question       0.50      0.76      0.60        25

          micro avg       0.47      0.61      0.53       211
          macro avg       0.25      0.33      0.27       211
       weighted avg       0.60      0.61      0.60       211
        samples avg       0.51      0.60      0.53       211



In [126]:
linear_svm(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.419
Precision: 0.636
Recall: 0.630
F_score: 0.617

                     precision    recall  f1-score   support

            sarcasm       0.80      0.70      0.74       138
              irony       0.21      0.31      0.25        36
             satire       0.09      0.80      0.16         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.06      0.50      0.11         6
rhetorical_question       0.61      0.76      0.68        25

          micro avg       0.40      0.63      0.49       211
          macro avg       0.30      0.51      0.32       211
       weighted avg       0.64      0.63      0.62       211
        samples avg       0.44      0.63      0.50       211



In [127]:
logistic_regression(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.379
Precision: 0.635
Recall: 0.588
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.81      0.59      0.68       138
              irony       0.23      0.47      0.31        36
             satire       0.07      0.40      0.11         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.09      0.83      0.17         6
rhetorical_question       0.53      0.76      0.62        25

          micro avg       0.37      0.59      0.45       211
          macro avg       0.29      0.51      0.32       211
       weighted avg       0.64      0.59      0.58       211
        samples avg       0.40      0.58      0.45       211



In [128]:
random_forest(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')



       Random Forest

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [129]:
gradient_boosting(train_features,y_train_l,test_features,y_test_l)


   Gradient Boosting

Accuracy: 0.600
Precision: 0.583
Recall: 0.607
F_score: 0.590

                     precision    recall  f1-score   support

            sarcasm       0.80      0.88      0.84       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.50      0.28      0.36        25

          micro avg       0.69      0.61      0.65       211
          macro avg       0.22      0.19      0.20       211
       weighted avg       0.58      0.61      0.59       211
        samples avg       0.66      0.64      0.64       211



In [130]:
from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler()
train_transformed=scaled_features.fit_transform(x_train_feat.values)
test_transformed=scaled_features.transform(x_test_feat.values)

In [131]:
train_features_transfomred = hstack([
    x_train_word_emb
    ,csr_matrix(np.array(train_transformed, dtype=float))], 'csr'
)

In [132]:
test_features_transfomred = hstack([
    x_test_word_emb
    ,csr_matrix(np.array(test_transformed, dtype=float))], 'csr'
)

In [133]:
linear_svm_grid(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.544
Precision: 0.618
Recall: 0.668
F_score: 0.636

                     precision    recall  f1-score   support

            sarcasm       0.80      0.79      0.79       138
              irony       0.24      0.31      0.27        36
             satire       0.15      0.40      0.22         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.14      0.50      0.22         6
rhetorical_question       0.41      0.64      0.50        25

          micro avg       0.54      0.67      0.60       211
          macro avg       0.29      0.44      0.33       211
       weighted avg       0.62      0.67      0.64       211
        samples avg       0.59      0.69      0.61       211



In [134]:
nonlinear_svm_grid(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [135]:
logistic_regression_grid(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.501
Precision: 0.608
Recall: 0.616
F_score: 0.606

                     precision    recall  f1-score   support

            sarcasm       0.79      0.74      0.76       138
              irony       0.18      0.22      0.20        36
             satire       0.12      0.20      0.15         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.08      0.33      0.13         6
rhetorical_question       0.46      0.68      0.55        25

          micro avg       0.49      0.62      0.54       211
          macro avg       0.27      0.36      0.30       211
       weighted avg       0.61      0.62      0.61       211
        samples avg       0.54      0.62      0.55       211



In [136]:
linear_svm(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.397
Precision: 0.656
Recall: 0.659
F_score: 0.642

                     precision    recall  f1-score   support

            sarcasm       0.81      0.73      0.77       138
              irony       0.24      0.33      0.28        36
             satire       0.06      0.80      0.11         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.05      0.50      0.09         6
rhetorical_question       0.70      0.76      0.73        25

          micro avg       0.37      0.66      0.47       211
          macro avg       0.31      0.52      0.33       211
       weighted avg       0.66      0.66      0.64       211
        samples avg       0.42      0.66      0.48       211



In [137]:
logistic_regression(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.377
Precision: 0.641
Recall: 0.621
F_score: 0.616

                     precision    recall  f1-score   support

            sarcasm       0.79      0.70      0.74       138
              irony       0.22      0.31      0.26        36
             satire       0.04      0.40      0.06         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.05      0.67      0.10         6
rhetorical_question       0.69      0.72      0.71        25

          micro avg       0.35      0.62      0.45       211
          macro avg       0.30      0.46      0.31       211
       weighted avg       0.64      0.62      0.62       211
        samples avg       0.40      0.63      0.46       211



In [138]:
random_forest(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')



       Random Forest

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [139]:
gradient_boosting(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l)


   Gradient Boosting

Accuracy: 0.608
Precision: 0.584
Recall: 0.611
F_score: 0.593

                     precision    recall  f1-score   support

            sarcasm       0.80      0.88      0.84       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.50      0.28      0.36        25

          micro avg       0.70      0.61      0.65       211
          macro avg       0.22      0.19      0.20       211
       weighted avg       0.58      0.61      0.59       211
        samples avg       0.67      0.65      0.64       211

